In [28]:
import pandas as pd

data_path = '../data/raw/online_retail.csv'
df = pd.read_csv(data_path)
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [30]:
df = df[~df['InvoiceNo'].str.startswith('C')]
df = df.dropna()
df = df[(df['UnitPrice']>0) & (df['Quantity']>0)]

rfm = df.groupby('CustomerID').agg(
    latest_date = ('InvoiceDate', 'max'),
    F = ('InvoiceNo', 'nunique'), #下单数量
    M = ('Quantity', 'sum') #购买产品数量
).reset_index()

customer_countries = df.drop_duplicates('CustomerID')[['CustomerID', 'Country']]

rfm['latest_date'] = pd.to_datetime(rfm['latest_date'])
# 多少天没有购买
rfm['R'] = rfm['latest_date'].apply(lambda x: rfm['latest_date'].max()-x)
rfm['R'] = rfm['R'].dt.days
rfm = rfm[['CustomerID', 'R', 'F', 'M']]
rfm = rfm.merge(customer_countries, on='CustomerID', how='left')
rfm

,CustomerID,R,F,M,Country
0,12346.0,325,1,74215,United Kingdom
1,12347.0,1,7,2458,Iceland
2,12348.0,74,4,2341,Finland
3,12349.0,18,1,631,Italy
4,12350.0,309,1,197,Norway
...,...,...,...,...,...
4332,18280.0,277,1,45,United Kingdom
4333,18281.0,180,1,54,United Kingdom
4334,18282.0,7,2,103,United Kingdom
4335,18283.0,3,16,1397,United Kingdom


In [31]:

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
scaler = StandardScaler()
rfm_nor = scaler.fit_transform(rfm[['R', 'F', 'M']])

kmeans = KMeans(n_clusters=4)
kmeans.fit(rfm_nor)
rfm['Cluster'] = kmeans.labels_
cluster_labels = kmeans.labels_
rfm['Cluster'] = cluster_labels
cluster_names = {
    0: 'new customer',
    1: 'lost customer',
    2: 'potential loyal customer',
    3: 'loyal customer'
}
rfm['Cluster'] = rfm['Cluster'].map(cluster_names)
rfm['Cluster'].value_counts()

/Users/a123/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Cluster
new customer                3025
loyal customer              1065
lost customer                229
potential loyal customer      18
Name: count, dtype: int64

In [32]:
rfm

,CustomerID,R,F,M,Country,Cluster
0,12346.0,325,1,74215,United Kingdom,potential loyal customer
1,12347.0,1,7,2458,Iceland,new customer
2,12348.0,74,4,2341,Finland,new customer
3,12349.0,18,1,631,Italy,new customer
4,12350.0,309,1,197,Norway,loyal customer
...,...,...,...,...,...,...
4332,18280.0,277,1,45,United Kingdom,loyal customer
4333,18281.0,180,1,54,United Kingdom,loyal customer
4334,18282.0,7,2,103,United Kingdom,new customer
4335,18283.0,3,16,1397,United Kingdom,lost customer


In [33]:
cluster_counts = rfm['Cluster'].value_counts()
potential_count = cluster_counts.get('potential loyal customer', 0)
lost_count = cluster_counts.get('lost customer', 0)
new_count = cluster_counts.get('new customer', 0)

print(f"Potential Loyal Customers: {potential_count}")
print(f"Lost Customers: {lost_count}")
print(f"New Customers: {new_count}")


Potential Loyal Customers: 18
Lost Customers: 229
New Customers: 3025


In [34]:
cluster_counts

Cluster
new customer                3025
loyal customer              1065
lost customer                229
potential loyal customer      18
Name: count, dtype: int64

In [35]:
rfm

,CustomerID,R,F,M,Country,Cluster
0,12346.0,325,1,74215,United Kingdom,potential loyal customer
1,12347.0,1,7,2458,Iceland,new customer
2,12348.0,74,4,2341,Finland,new customer
3,12349.0,18,1,631,Italy,new customer
4,12350.0,309,1,197,Norway,loyal customer
...,...,...,...,...,...,...
4332,18280.0,277,1,45,United Kingdom,loyal customer
4333,18281.0,180,1,54,United Kingdom,loyal customer
4334,18282.0,7,2,103,United Kingdom,new customer
4335,18283.0,3,16,1397,United Kingdom,lost customer


In [36]:
rfm = pd.DataFrame(rfm)
rfm.to_csv('../data/processed/processed_rfm_model.csv', index=False)


In [15]:
rfm

,CustomerID,R,F,M,Country,Cluster
0,12346.0,325,1,74215,United Kingdom,loyal customer
1,12347.0,1,7,2458,Iceland,new customer
2,12348.0,74,4,2341,Finland,new customer
3,12349.0,18,1,631,Italy,new customer
4,12350.0,309,1,197,Norway,lost customer
...,...,...,...,...,...,...
4333,18280.0,277,1,45,United Kingdom,lost customer
4334,18281.0,180,1,54,United Kingdom,lost customer
4335,18282.0,7,2,103,United Kingdom,new customer
4336,18283.0,3,16,1397,United Kingdom,potential loyal customer
